####  Connectivity stuff ..

In [ ]:
#  From,
#
#     https://github.com/KatanaGraph/katana-enterprise/blob/master/python/test/notebooks/red/analytics_remote.ipynb


In [1]:
# import os

from katana import remote
from katana.remote import import_data


# os.environ["KATANA_SERVER_ADDRESS"] = "localhost:8080"

my_client = remote.Client()

print(my_client)

VERBOSE: ../../../../external/katana/libgalois/include/katana/AsyncPool.h:152: 12983: async pool tids: 140042377660160, 140042291414784
VERBOSE: ../../../../external/katana/libgalois/include/katana/AsyncPool.h:152: 12983: async pool tids: 140042289313536, 140042285115136
VERBOSE: ../../../../external/katana/libgalois/include/katana/AsyncPool.h:152: 12983: async pool tids: 140042280916736, 140042272524032
VERBOSE: ../../../../external/katana/libgalois/include/katana/AsyncPool.h:152: 12983: async pool tids: 140042264131328, 140042247350016
VERBOSE: ../../../../external/katana/libgalois/include/katana/AsyncPool.h:152: 12983: async pool tids: 140042230568704, 140042228467456


In [2]:
DB_NAME    = "my_db"
GRAPH_NAME = "my_graph"

print("--")

--


In [ ]:
#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")

In [ ]:
#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)

In [ ]:
#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)

In [ ]:
#  CREATE GRAPH
#
my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=2)

print(my_graph)

In [3]:
#  CONNECT TO GRAPH

for l_graph in my_client.get_database(name=DB_NAME).graphs_in_database():
   if (l_graph.name == GRAPH_NAME):
      my_graph=my_client.get_database(name=DB_NAME).get_graph_by_id(id=l_graph.graph_id)
         #
      break

print(my_graph)

<_Graph my_graph, 2hbu4gHU6ycW7cmt2SWQ1dBeN9qUGZ2nvfygQFNQNpx6, 0>


####  Libraries specfic to Analytics ..

In [4]:

import numpy as np
from katana import remote
from katana.remote import analytics, import_data
from katana_enterprise.distributed import PartitionPolicy
# from util import notebook_config

print("--")


--


####  Actual analytics

In [ ]:

#  This was code from the online example; I don't run this ..
# 
# graph_path = nbconf.get_rdg_dataset_path(
#    "rmat10_symmetric", "gs://katana-demo-datasets/rdg-datasets/v3/rmat10_symmetric"
# )
# 
# import_data.rdg(graph, graph_path)


In [ ]:

#  This was code from the online example; I don't run this ..
# 
# prop_name           = unique_name("sssp_dist")
# source              = nbconf.choice(0, [0, 1, 2, 3, 4, 5, 6, 8, 9])
# sssp_max_iterations = nbconf.range_int(1, min=1, max=2, step=1)
# sssp_delta          = nbconf.range_int(0, min=0, max=10, step=1)
# 
# 
# analytics.sssp(
#     graph,
#     weight_property="value",
#     source=source,
#     result_property_name=prop_name,
#     max_iterations=sssp_max_iterations,
#     delta=sssp_delta,
# )
# 
# result = graph.query(
#     f"""
#     MATCH (n) 
#     RETURN n, n.{prop_name}
#     ORDER BY n.{prop_name}
#     LIMIT 30
#     """,
#     contextualize=True,
# )
# 
# for node_sssp in result[f"n.{prop_name}"].values:
#     node_sssp = node_sssp[0]
#     assert node_sssp == 0
# 
# result.view()


In [ ]:


l_prop_name           = "airport_code" # "bob"
l_source              = 2
l_sssp_max_iterations = 5 # 1000 # 5
l_sssp_delta          = 0


analytics.sssp(
    my_graph,
    weight_property      = "LENGTH",
    source               = l_source,
    result_property_name = l_prop_name,
    max_iterations       = l_sssp_max_iterations,
    delta                = l_sssp_delta,
)

print("--")


          0/? [?op/s]

In [ ]:

l_result = my_graph.query(
    """
    MATCH (n) 
    RETURN n // , n.{l_prop_name}
    // ORDER BY n.{l_prop_name}
    // LIMIT 30
    """,
    # contextualize=True,
   )




In [ ]:
# ##################################################################